In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as F
from sklearn.metrics import f1_score

# Step 1: Data Augmentation and Loading
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

dataset_dir = 'dataset'  # Replace with your dataset directory
full_dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)

# Split dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = nn.ReLU(inplace=True)(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = nn.ReLU(inplace=True)(out)

        return out

class custom(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(custom, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.in_planes, planes, stride, downsample))
        self.in_planes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def model_diffusion(num_classes=1000):
    return custom(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Instantiate the model
num_classes = len(full_dataset.classes)  # Number of classes in your dataset
model = model_diffusion(num_classes=num_classes)

# Load the saved model
model.load_state_dict(torch.load('92%_custom_model_diffusion.pth'))
model.eval()  # Set the model to evaluation mode

# Define the test dataset and data loader
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust if your model uses a different input size
    transforms.ToTensor()
])

# Collect predictions and true labels
all_preds = []
all_labels = []

# Add tqdm for progress tracking in the loop
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating", leave=True):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)  # Get predicted class
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate the F1 score
f1 = f1_score(all_labels, all_preds, average='weighted')  # Choose 'macro', 'micro', or 'weighted' depending on your needs
print(f"F1 Score: {f1}")

/var/folders/g1/jh2048gn2tx119xcnmnxlg4m0000gn/T/ipykernel_13841/1266406122.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('92%_custo

F1 Score: 0.9428009242668879


In [3]:
import os
import warnings
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

base_dir = 'Dataset'
# Suppress TensorFlow and Keras logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Suppress specific Keras warnings
warnings.filterwarnings("ignore", category=UserWarning, module='keras.src.trainers.data_adapters.py_dataset_adapter')

# Load the saved model and compile it
model = load_model('plant_disease_classifier_fast_cnn.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming the test dataset is organized in a directory
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    base_dir,  # Replace with your actual test dataset directory
    target_size=(64, 64),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

# Predict the output for test data
y_pred = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)

# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from test_generator
y_true = test_generator.classes

# Calculate the F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"F1 Score: {f1}")

Found 20638 images belonging to 15 classes.
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step
F1 Score: 0.7112471774948349


In [1]:
import torch
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Set the path to your dataset
dataset_dir = 'dataset'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


# Load the full dataset
full_dataset = ImageFolder(root=dataset_dir, transform=transform)

# Split the dataset into training and test sets (80% train, 20% test)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 1: Define your model architecture
# Replace this with your actual model class definition
import torch.nn as nn
import torch.optim as optim

class DiffusionModelWithClassification(nn.Module):
    def __init__(self, num_classes):
        super(DiffusionModelWithClassification, self).__init__()
        # Encoder with additional layers and batch normalization
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        # Decoder remains the same
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid(),
        )

        # Classifier with additional layers and dropout
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 16 * 16, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.encoder(x)
        features = self.decoder(x)
        x = self.classifier(x)
        return features, x

# Number of classes (based on the number of subfolders)
num_classes = len(full_dataset.classes)
model = DiffusionModelWithClassification(num_classes=num_classes)

# Step 2: Load the state dictionary
state_dict = torch.load('30%_accuracy.pth')
# Remove the mismatched layers from the state_dict
state_dict.pop('decoder.0.weight')
state_dict.pop('decoder.0.bias')
state_dict.pop('classifier.1.weight')
state_dict.pop('classifier.1.bias')

# Load the remaining layers
model.load_state_dict(state_dict, strict=False)

# Step 3: Set the model to evaluation mode
model.eval()

# Assuming your test_loader is already defined in the notebook
# Extract true labels and predictions
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating the model"):
        # Get the classification output (the second element in the tuple)
        _, outputs = model(images)
        
        # Find the class with the maximum score
        _, predicted = torch.max(outputs, 1)
        
        # Append true labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calculate the F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"F1 Score: {f1}")

/var/folders/g1/jh2048gn2tx119xcnmnxlg4m0000gn/T/ipykernel_22334/2235203144.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('30%_accuracy.pth')


F1 Score: 0.019481291722944636


In [3]:
from sklearn.metrics import f1_score
import torch
from torchvision import models
from tqdm import tqdm

# Load the saved model with the correct output size
model = models.resnet18()
num_classes = 15  # Adjust this based on your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Modify the fully connected layer

# Load the saved model state
model.load_state_dict(torch.load('98%_accuracy.pth'))
model.eval()  # Set the model to evaluation mode

true_labels = []
pred_labels = []

# Assuming you have your test_loader defined
# Disable gradient calculation for evaluation
with torch.no_grad():
    # Wrap test_loader with tqdm for progress tracking
    for inputs, labels in tqdm(test_loader, desc="Evaluating Model"):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        pred_labels.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate F1 score
f1 = f1_score(true_labels, pred_labels, average='weighted')  # or 'macro' for multi-class
print(f"F1 Score: {f1}")

/var/folders/g1/jh2048gn2tx119xcnmnxlg4m0000gn/T/ipykernel_22334/648743093.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('98%_accurac

F1 Score: 0.9929646127790567
